In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import numpy as np
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn 
from torch import optim

In [2]:
class TMDDataset(Dataset):
    def __init__(self, csv_file,transform=None):
        self.data_frame=pd.read_csv(csv_file)
        self.labels = np.asarray(self.data_frame.iloc[:, -1])
        self.transform = transform
        
    def __len__(self):
        return len(self.data_frame)   
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        sample_x =  np.asarray(self.data_frame.iloc[idx, :-1])
        sample_y = np.asarray(self.data_frame.iloc[idx, -1])
        
        
        if self.transform:
            sample_x = self.transform(sample_x)

            
        sample = (sample_x,  sample_y)  
        return sample      
    

In [3]:
tmd_train = TMDDataset('train.csv', transform = transforms.Compose([transforms.ToTensor()]))

In [4]:
tmd_test = TMDDataset('test.csv' , transform = transforms.Compose([transforms.ToTensor()]))

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=tmd_train,
                                                    batch_size=64,
                                                    shuffle=False)

In [6]:
test_loader = torch.utils.data.DataLoader(dataset=tmd_test,
                                                    batch_size=64,
                                                    shuffle=False)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
input_size = 37
hidden_sizes = [800, 512 ]
output_size = 5


In [32]:
class SyNet_client(nn.Module):
    def __init__(self):
        super(SyNet_client, self).__init__()
        
        self.lin = nn.Linear(input_size, hidden_sizes[0])
        self.lin2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.dropout = nn.Dropout(p=0.2)
        
        
        
    def forward(self, x):
        x = self.lin(x)
        x = torch.nn.functional.relu(x)
        x = self.dropout(x)
        x = self.lin2(x)
        x = torch.nn.functional.relu(x)
        x = self.dropout(x)
        return x
    
class SyNet_server(nn.Module):
    def __init__(self):
        super(SyNet_server, self).__init__()
        self.server = nn.Sequential(nn.Linear(hidden_sizes[1], hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Dropout(p=0.2),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Dropout(p=0.2),
                      nn.Linear(hidden_sizes[1], hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Dropout(p=0.2),
                      nn.Linear(hidden_sizes[0], output_size),
                      nn.LogSoftmax(dim=1) )
        
    def forward(self, x):
        x = self.server(x)
        return x

In [47]:
model1 = SyNet_client()

In [48]:
opt1 = torch.optim.SGD(params=model1.parameters(),lr=0.003)

In [49]:
#Model 2
model2 = SyNet_server()
opt2 = torch.optim.SGD(params=model2.parameters(),lr=0.003)

In [50]:
criterion = nn.NLLLoss()

In [51]:
model1.cuda(device)

SyNet_client(
  (lin): Linear(in_features=37, out_features=800, bias=True)
  (lin2): Linear(in_features=800, out_features=512, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [52]:
model2.cuda(device)

SyNet_server(
  (server): Sequential(
    (0): Linear(in_features=512, out_features=800, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=800, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=512, out_features=800, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=800, out_features=5, bias=True)
    (10): LogSoftmax(dim=1)
  )
)

In [ ]:
train_accuracies , test_accuracies, train_losses, test_losses = [], [], [], []
test_accuracy_checkpt=0
model1.train()
model2.train()

epochs = 500
for e in range(epochs):
    train_loss = 0
    train_accuracy = 0
    
    for feature, labels in train_loader:

        feature, labels = feature.to(device), labels.to(device)
        opt1.zero_grad()
        opt2.zero_grad()
        
        activation= model1.forward(feature.float())
        activation.retain_grad()
        log_ps = model2(activation)
        loss = criterion(log_ps, labels)
        loss.backward(retain_graph=True) #loss.backward(retain_graph=True)

        activation_grad = activation.grad
        activation.backward(activation_grad,retain_graph=True) #activation.backward(activation_grad,retain_graph=True)
        opt1.step()
        opt2.step()
        
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        
        equals = top_class == labels.view(*top_class.shape)
        train_accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        train_loss += loss.item()
        
    else:
        
        test_loss = 0
        test_accuracy = 0
        model1.eval()
        model2.eval()
        
        with torch.no_grad():
            
            for feature, labels in test_loader:
                
                feature, labels = feature.to(device), labels.to(device)
                
                activation= model1.forward(feature.float())
                log_ps = model2(activation)
                test_loss += criterion(log_ps, labels).item()
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                test_accuracy += torch.mean(equals.type(torch.FloatTensor))
         
        model1.train()
        model2.train()
        train_accuracies.append(train_accuracy/len(train_loader))
        test_accuracies.append(test_accuracy/len(test_loader))
        train_losses.append(train_loss/len(train_loader))
        test_losses.append(test_loss/len(test_loader))
        
        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Accuracy: {:.3f}.. ".format(train_accuracies[-1]),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Test Accuracy: {:.3f}..".format(test_accuracies[-1]),
              "Test Loss: {:.3f}.. ".format(test_losses[-1])
        )
        if test_accuracy_checkpt < test_accuracies [-1]:
            torch.save(model1.state_dict(), './model1.pth')
            torch.save(model2.state_dict(), './model2.pth')
            test_accuracy_checkpt = test_accuracies [-1]
            print("check point epoch : {}".format(e+1) )

In [58]:
test_model1=SyNet_client()

In [59]:
#Model 2
test_model2 = SyNet_server()

In [60]:
test_model1.cuda(device)

SyNet_client(
  (lin): Linear(in_features=37, out_features=800, bias=True)
  (lin2): Linear(in_features=800, out_features=512, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [61]:
test_model2.cuda(device)

SyNet_server(
  (server): Sequential(
    (0): Linear(in_features=512, out_features=800, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=800, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=512, out_features=800, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=800, out_features=5, bias=True)
    (10): LogSoftmax(dim=1)
  )
)

In [62]:
state_dict_1 = torch.load('./model1.pth')

In [63]:
state_dict_2 = torch.load('./model2.pth')

In [64]:
state_dict_1

OrderedDict([('lin.weight',
              tensor([[ 0.0159,  0.1094,  0.1541,  ..., -0.1280,  0.0520, -0.1170],
                      [-0.0729,  0.0043, -0.1454,  ..., -0.0813,  0.0970,  0.1421],
                      [ 0.0304, -0.1586,  0.1059,  ...,  0.0860, -0.1520,  0.0014],
                      ...,
                      [-0.0834,  0.1619, -0.1028,  ..., -0.1638, -0.1419,  0.1840],
                      [ 0.1597,  0.0628, -0.1111,  ...,  0.1919,  0.0828, -0.0546],
                      [ 0.0201,  0.2183, -0.1363,  ..., -0.1256,  0.0253,  0.0138]],
                     device='cuda:0')),
             ('lin.bias',
              tensor([ 0.0064,  0.0809, -0.0691, -0.1763, -0.0425,  0.1038, -0.0566, -0.1874,
                       0.1312, -0.0564,  0.0658, -0.0531, -0.0494, -0.1389, -0.0446,  0.0844,
                      -0.0076,  0.0260, -0.2492,  0.0187,  0.1289, -0.1601,  0.1377,  0.0492,
                      -0.2018,  0.1311, -0.1191, -0.0771,  0.0899, -0.1255,  0.0800, -0.1228

In [66]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report

In [67]:
x_test_ch = torch.from_numpy(np.asarray(pd.read_csv('test.csv').iloc[:,:-1])).to(device).float()

In [68]:
y_test_ch = torch.from_numpy(np.asarray(pd.read_csv('test.csv').iloc[:,-1])).to(device)

In [69]:
test_model1.load_state_dict(state_dict_1)

<All keys matched successfully>

In [70]:
test_model2.load_state_dict(state_dict_2)

<All keys matched successfully>

In [71]:
test_model1.eval()
test_model2.eval()
with torch.no_grad():
    activation = test_model1.forward(x_test_ch)
    log_ps = test_model2.forward(activation)
log_ps

tensor([[-2.1005e+01,  0.0000e+00, -3.4697e+01, -1.9301e+01, -2.4057e+01],
        [-1.9012e+01, -6.7471e+00, -7.3251e+00, -1.8352e-03, -1.4775e+01],
        [-9.4208e+00, -5.3068e+00, -6.7312e+00, -6.2732e-03, -1.0731e+01],
        ...,
        [-1.5157e+01, -1.1844e+01, -5.2260e-04, -1.0738e+01, -7.6142e+00],
        [-1.4502e-01, -2.0050e+00, -1.2814e+01, -9.0915e+00, -8.4069e+00],
        [-4.7746e+00, -4.9451e+00, -4.4475e-01, -7.7988e+00, -1.0699e+00]],
       device='cuda:0')

In [72]:
ps = torch.exp(log_ps)

In [73]:
top_p, top_class = ps.topk(1, dim=1)

In [74]:
print(classification_report(y_test_ch.cpu(), top_class.cpu()))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       255
           1       0.97      0.90      0.93       228
           2       0.96      0.93      0.94       239
           3       0.94      0.96      0.95       239
           4       0.96      0.95      0.96       218

    accuracy                           0.94      1179
   macro avg       0.95      0.94      0.94      1179
weighted avg       0.95      0.94      0.94      1179



<p style="text-align:center">
    <img src="train.png" width="600">
</p>

<p style="text-align:center">
    <img src="test.png" width="600">
</p>